In [ ]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"sibbir","key":"c9b1f65daec00b23325e5e866738f3b6"}'}

In [ ]:
import os
import zipfile

# Create a Kaggle directory
os.makedirs("/root/.kaggle", exist_ok=True)

# Move kaggle.json to the correct directory
!mv kaggle.json /root/.kaggle/

# Set correct permissions
!chmod 600 /root/.kaggle/kaggle.json

In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("shuvoalok/raf-db-dataset")

print("Path to dataset files:", path)

100%|██████████| 37.7M/37.7M [00:01<00:00, 26.0MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/shuvoalok/raf-db-dataset/versions/2


In [ ]:
import os
import torch
import torch.optim as optim
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import timm
from torch.utils.data import DataLoader
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import matplotlib.pyplot as plt
from tqdm import tqdm

In [ ]:
import pandas as pd

train_csv_path = "/root/.cache/kagglehub/datasets/shuvoalok/raf-db-dataset/versions/2/train_labels.csv"
test_csv_path = "/root/.cache/kagglehub/datasets/shuvoalok/raf-db-dataset/versions/2/test_labels.csv"

# Load CSV files
train_labels = pd.read_csv(train_csv_path)
test_labels = pd.read_csv(test_csv_path)

# Show first few rows
print("Train CSV Sample:")
print(train_labels.head())

print("\nTest CSV Sample:")
print(test_labels.head())

Train CSV Sample:
                     image  label
0  train_00001_aligned.jpg      5
1  train_00002_aligned.jpg      5
2  train_00003_aligned.jpg      4
3  train_00004_aligned.jpg      4
4  train_00005_aligned.jpg      5

Test CSV Sample:
                   image  label
0  test_0001_aligned.jpg      5
1  test_0002_aligned.jpg      1
2  test_0003_aligned.jpg      4
3  test_0004_aligned.jpg      1
4  test_0005_aligned.jpg      5


In [ ]:
# Print all unique labels from the train and test datasets
print("Unique labels in train dataset:", train_labels["label"].unique())
print("Unique labels in test dataset:", test_labels["label"].unique())

Unique labels in train dataset: [5 4 1 6 2 3 7]
Unique labels in test dataset: [5 1 4 3 6 2 7]


In [ ]:
import os

dataset_path = "/root/.cache/kagglehub/datasets/shuvoalok/raf-db-dataset/versions/2/DATASET"

# List a few sample files from the dataset directory
sample_files = os.listdir(dataset_path)[:10]
print("Sample files in dataset directory:", sample_files)

Sample files in dataset directory: ['train', 'test']


In [ ]:
import os

train_path = "/root/.cache/kagglehub/datasets/shuvoalok/raf-db-dataset/versions/2/DATASET/train"
test_path = "/root/.cache/kagglehub/datasets/shuvoalok/raf-db-dataset/versions/2/DATASET/test"

# Check inside one of the numbered folders
for label in os.listdir(train_path):
    label_folder = os.path.join(train_path, label)
    if os.path.isdir(label_folder):  # Ensure it's a folder
        print(f"Sample images in {label}:", os.listdir(label_folder)[:10])

Sample images in 3: ['train_09634_aligned.jpg', 'train_03605_aligned.jpg', 'train_09738_aligned.jpg', 'train_09675_aligned.jpg', 'train_09499_aligned.jpg', 'train_09646_aligned.jpg', 'train_03989_aligned.jpg', 'train_09409_aligned.jpg', 'train_04311_aligned.jpg', 'train_01874_aligned.jpg']
Sample images in 4: ['train_03528_aligned.jpg', 'train_04784_aligned.jpg', 'train_03592_aligned.jpg', 'train_02812_aligned.jpg', 'train_01942_aligned.jpg', 'train_06397_aligned.jpg', 'train_09067_aligned.jpg', 'train_08352_aligned.jpg', 'train_02918_aligned.jpg', 'train_00310_aligned.jpg']
Sample images in 5: ['train_00835_aligned.jpg', 'train_06008_aligned.jpg', 'train_01180_aligned.jpg', 'train_01088_aligned.jpg', 'train_07217_aligned.jpg', 'train_04418_aligned.jpg', 'train_04197_aligned.jpg', 'train_07984_aligned.jpg', 'train_07829_aligned.jpg', 'train_01366_aligned.jpg']
Sample images in 7: ['train_09898_aligned.jpg', 'train_09826_aligned.jpg', 'train_10034_aligned.jpg', 'train_10329_aligned.jpg'

In [ ]:
import os
import shutil
import pandas as pd

# Define correct paths
dataset_path_train = "/root/.cache/kagglehub/datasets/shuvoalok/raf-db-dataset/versions/2/DATASET/train"
dataset_path_test = "/root/.cache/kagglehub/datasets/shuvoalok/raf-db-dataset/versions/2/DATASET/test"

train_csv_path = "/root/.cache/kagglehub/datasets/shuvoalok/raf-db-dataset/versions/2/train_labels.csv"
test_csv_path = "/root/.cache/kagglehub/datasets/shuvoalok/raf-db-dataset/versions/2/test_labels.csv"

train_dir = "/root/rafdb/train"
test_dir = "/root/rafdb/test"

# Load CSV labels
train_labels = pd.read_csv(train_csv_path)
test_labels = pd.read_csv(test_csv_path)

# Define label mapping
label_map = {
    1: "Happy",
    2: "Sad",
    3: "Surprise",
    4: "Fear",
    5: "Disgust",
    6: "Anger",
    7: "Contempt"
}

# Create class directories
for label in label_map.values():
    os.makedirs(os.path.join(train_dir, label), exist_ok=True)
    os.makedirs(os.path.join(test_dir, label), exist_ok=True)

print("Class folders created successfully.")

import os
import shutil
from pathlib import Path

def move_images(labels_df, source_base_dir, target_base_dir, label_map):
    """Move images based on labels dataframe to organized class directories."""
    for _, row in labels_df.iterrows():
        image_name = row["image"]
        label_id = row["label"]

        if label_id not in label_map:
            continue

        class_name = label_map[label_id]
        src = Path(source_base_dir) / str(label_id) / image_name
        dest = Path(target_base_dir) / class_name / image_name

        # Create target directory if it doesn't exist
        dest.parent.mkdir(parents=True, exist_ok=True)

        if src.exists():
            shutil.move(str(src), str(dest))
        else:
            print(f"Warning: {src} not found!")

# Move train images
move_images(train_labels, dataset_path_train, train_dir, label_map)

# Move test images
move_images(test_labels, dataset_path_test, test_dir, label_map)

print("Dataset successfully organized.")

Class folders created successfully.
Dataset successfully organized.


In [ ]:
# Data preprocessing and augmentation
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ToTensor(),
    transforms.Normalize([0.5], [0.5])
])

In [ ]:
# Define Model
class CustomViT(nn.Module):
    def __init__(self, num_classes=7):
        super(CustomViT, self).__init__()
        self.model = timm.create_model("vit_large_patch16_224", pretrained=True, num_classes=num_classes)

    def forward(self, x):
        return self.model(x)

In [ ]:
import torch
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.utils.data import DataLoader

# Define transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

# Load dataset
train_dir = "/root/rafdb/train"
test_dir = "/root/rafdb/test"

train_dataset = datasets.ImageFolder(train_dir, transform=transform)
test_dataset = datasets.ImageFolder(test_dir, transform=transform)

# Use batch size 8 to prevent crashes
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True, num_workers=2)
test_loader = DataLoader(test_dataset, batch_size=8, shuffle=False, num_workers=2)

# Print final class labels
class_names = train_dataset.classes
print(f"Classes used for training: {class_names}")

Classes used for training: ['Anger', 'Contempt', 'Disgust', 'Fear', 'Happy', 'Sad', 'Surprise']


In [ ]:
import timm
import torch
import torch.nn as nn
from torch.cuda.amp import autocast, GradScaler

# Check for GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load ConvNeXt Large
model = timm.create_model("convnext_large", pretrained=True, num_classes=len(class_names))
model.to(device)

# Enable mixed precision
scaler = GradScaler()

# Define loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4)

print("ConvNeXt model loaded with mixed precision enabled.")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model.safetensors:   0%|          | 0.00/791M [00:00<?, ?B/s]

ConvNeXt model loaded with mixed precision enabled.


<ipython-input-19-81b482fe9985>:14: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = GradScaler()


In [ ]:
!pip install onnx onnxruntime

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 106.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 84.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 9.0 MB/s eta 0:00:00


In [ ]:
import torch
import torch.optim as optim
import torch.nn as nn
import os
import timm
from tqdm import tqdm
from typing import Tuple, Dict
import datetime
# Importing necessary modules
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.utils.data import DataLoader

# Configuration Constants
CONFIG = {
    "checkpoint_dir": "/content/checkpoints/",  # Directory to save checkpoints
    "num_classes": 7,
    "model_name": "convnext_large",
    "batch_size": 32,
    "learning_rate": 1e-4,
    "weight_decay": 5e-4,
    "num_epochs": 20,
    "accumulation_steps": 2,
    "save_interval": 5,  # Save every 5 epochs
    "mixed_precision": True  # Use AMP for faster training
}

# Ensure checkpoint directory exists
os.makedirs(CONFIG["checkpoint_dir"], exist_ok=True)

# Define Model
class CustomConvNeXt(nn.Module):
    def __init__(self, num_classes: int = CONFIG["num_classes"]):
        super().__init__()
        self.model = timm.create_model(
            CONFIG["model_name"],
            pretrained=False,  # No pretrained weights
            num_classes=num_classes
        )

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        return self.model(x)

def get_device() -> torch.device:
    """Get the best available device."""
    if torch.cuda.is_available():
        return torch.device("cuda")
    elif torch.backends.mps.is_available():
        return torch.device("mps")
    return torch.device("cpu")

def initialize_model(device: torch.device) -> nn.Module:
    """Initialize model with proper weight initialization."""
    model = CustomConvNeXt().to(device)

    # Initialize weights properly
    def _init_weights(m):
        if isinstance(m, nn.Linear):
            nn.init.trunc_normal_(m.weight, std=0.02)
            if m.bias is not None:
                nn.init.zeros_(m.bias)

    model.apply(_init_weights)
    return model

def save_checkpoint(
    model: nn.Module,
    optimizer: optim.Optimizer,
    epoch: int,
    loss: float,
    path: str
) -> None:
    """Save model checkpoint."""
    torch.save({
        'epoch': epoch,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'loss': loss,
    }, path)
    print(f"Checkpoint saved to {path}")

def train_epoch(
    model: nn.Module,
    train_loader: torch.utils.data.DataLoader,
    criterion: nn.Module,
    optimizer: optim.Optimizer,
    device: torch.device,
    accumulation_steps: int,
    epoch: int
) -> Tuple[float, float]:
    """Train model for one epoch."""
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    optimizer.zero_grad()

    with tqdm(train_loader, desc=f"Epoch {epoch} Training", leave=False) as pbar:
        for batch_idx, (images, labels) in enumerate(pbar):
            images, labels = images.to(device), labels.to(device)

            # Mixed precision training
            with torch.amp.autocast(device_type=device.type, enabled=CONFIG["mixed_precision"]):
                outputs = model(images)
                loss = criterion(outputs, labels) / accumulation_steps

            loss.backward()

            if (batch_idx + 1) % accumulation_steps == 0:
                optimizer.step()
                optimizer.zero_grad()

            running_loss += loss.item() * accumulation_steps
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()

            pbar.set_postfix({
                'loss': running_loss / (batch_idx + 1),
                'acc': 100. * correct / total
            })

    train_loss = running_loss / len(train_loader)
    train_acc = 100.0 * correct / total
    return train_loss, train_acc

def validate(
    model: nn.Module,
    test_loader: torch.utils.data.DataLoader,
    criterion: nn.Module,
    device: torch.device
) -> Tuple[float, float]:
    """Validate model performance."""
    model.eval()
    val_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad(), tqdm(test_loader, desc="Validating", leave=False) as pbar:
        for images, labels in pbar:
            images, labels = images.to(device), labels.to(device)

            outputs = model(images)
            loss = criterion(outputs, labels)

            val_loss += loss.item()
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()

            pbar.set_postfix({
                'val_loss': val_loss / (pbar.n + 1),
                'val_acc': 100. * correct / total
            })

    val_loss /= len(test_loader)
    val_acc = 100.0 * correct / total
    return val_loss, val_acc

def train_model(
    model: nn.Module,
    train_loader: torch.utils.data.DataLoader,
    test_loader: torch.utils.data.DataLoader,
    criterion: nn.Module,
    optimizer: optim.Optimizer,
    device: torch.device,
    num_epochs: int = CONFIG["num_epochs"]
) -> Dict[str, list]:
    """Main training loop."""
    history = {
        'train_loss': [],
        'train_acc': [],
        'val_loss': [],
        'val_acc': []
    }

    start_time = datetime.datetime.now()
    print(f"Training started at {start_time.strftime('%Y-%m-%d %H:%M:%S')}")
    print(f"Training on {device} with config: {CONFIG}")

    for epoch in range(num_epochs):
        # Train phase
        train_loss, train_acc = train_epoch(
            model, train_loader, criterion, optimizer, device,
            CONFIG["accumulation_steps"], epoch
        )

        # Validation phase
        val_loss, val_acc = validate(model, test_loader, criterion, device)

        # Update history
        history['train_loss'].append(train_loss)
        history['train_acc'].append(train_acc)
        history['val_loss'].append(val_loss)
        history['val_acc'].append(val_acc)

        # Print epoch summary
        print(f"Epoch {epoch + 1}/{num_epochs} - "
              f"Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.2f}% - "
              f"Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.2f}%")

        # Save checkpoint
        if (epoch + 1) % CONFIG["save_interval"] == 0 or (epoch + 1) == num_epochs:
            checkpoint_path = os.path.join(
                CONFIG["checkpoint_dir"],
                f"checkpoint_epoch_{epoch + 1}.pt"
            )
            save_checkpoint(model, optimizer, epoch + 1, val_loss, checkpoint_path)

    training_time = datetime.datetime.now() - start_time
    print(f"Training completed in {training_time}")
    return history

def main():
    # Initialize
    device = get_device()
    print(f"Using device: {device}")

    model = initialize_model(device)

    # Define loss and optimizer
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.AdamW(
        model.parameters(),
        lr=CONFIG["learning_rate"],
        weight_decay=CONFIG["weight_decay"]
    )

    # Define transformations
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.RandomHorizontalFlip(),
        transforms.RandomRotation(10),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
    ])

    # Load dataset # Moving the dataset loading logic inside main()
    train_dir = "/root/rafdb/train"
    test_dir = "/root/rafdb/test"

    train_dataset = datasets.ImageFolder(train_dir, transform=transform)
    test_dataset = datasets.ImageFolder(test_dir, transform=transform)

    # Use batch size 8 to prevent crashes
    train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True, num_workers=2)
    test_loader = DataLoader(test_dataset, batch_size=8, shuffle=False, num_workers=2)

    # Train the model
    history = train_model(
        model, train_loader, test_loader, criterion, optimizer, device
    )

    # Save final model
    final_model_path = os.path.join(CONFIG["checkpoint_dir"], "final_model.pt")
    torch.save(model.state_dict(), final_model_path)
    print(f"Final model saved to {final_model_path}")

if __name__ == "__main__":
    main()

Using device: cuda


FileNotFoundError: [Errno 2] No such file or directory: '/root/rafdb/train'

In [ ]:
import matplotlib.pyplot as plt

# Training and Validation Accuracy Data
epochs = list(range(6, 21))
train_acc = [97.28, 98.07, 98.18, 98.70, 98.61, 98.81, 99.07, 98.78, 98.92, 99.10, 98.97, 98.97, 99.16, 99.32, 99.10]
val_acc = [85.72, 86.83, 86.18, 86.15, 85.72, 85.23, 86.90, 83.87, 84.62, 85.79, 85.56, 83.74, 86.67, 86.38, 85.82]

# Plot Training and Validation Accuracy
plt.figure(figsize=(10, 5))
plt.plot(epochs, train_acc, label="Train Accuracy", marker='o', linestyle='-')
plt.plot(epochs, val_acc, label="Validation Accuracy", marker='s', linestyle='--')

# Labels and Title
plt.xlabel("Epochs")
plt.ylabel("Accuracy (%)")
plt.title("Training vs Validation Accuracy Over Epochs")
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
import torch
import torch.nn as nn
import timm
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Dataset
import os
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from PIL import Image
import glob
import matplotlib.pyplot as plt
from tqdm import tqdm  # For progress bars
from typing import Dict, List, Tuple  # For type hints

# Constants (configurable parameters)
MODEL_PATH = "/content/model_epoch_20.pt"
TEST_LABELS_PATH = "/root/.cache/kagglehub/datasets/shuvoalok/raf-db-dataset/versions/2/test_labels.csv"
TEST_FOLDER = "/root/rafdb/test/"
BATCH_SIZE = 32  # Increased batch size for better GPU utilization
IMAGE_SIZE = 224
NUM_CLASSES = 7
NUM_WORKERS = min(4, os.cpu_count())  # Optimal number of workers for data loading

# Set device with fallback to MPS for Apple Silicon if available
device = torch.device(
    "cuda" if torch.cuda.is_available()
    else "mps" if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using device: {device}")

# Define label mapping as constant to avoid recreation
LABEL_MAP = {
    1: "Happy",
    2: "Sad",
    3: "Surprise",
    4: "Fear",
    5: "Disgust",
    6: "Anger",
    7: "Contempt"
}
REVERSE_LABEL_MAP = {v: k-1 for k, v in LABEL_MAP.items()}  # For faster label to index conversion

class CustomConvNeXt(nn.Module):
    """Custom ConvNeXt model with pretrained weights."""
    def __init__(self, num_classes: int = NUM_CLASSES):
        super().__init__()  # More modern syntax
        self.model = timm.create_model(
            "convnext_large",
            pretrained=False,
            num_classes=num_classes
        )

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        return self.model(x)

def load_model(model_path: str) -> nn.Module:
    """Load and prepare the model for evaluation."""
    if not os.path.exists(model_path):
        raise FileNotFoundError(f"Model file not found: {model_path}")

    model = CustomConvNeXt(num_classes=NUM_CLASSES).to(device)

    # Load with error handling
    try:
        model = torch.jit.load(model_path, map_location=device)
        model.eval()
        print(f"Model successfully loaded from: {model_path}")
        return model
    except Exception as e:
        raise RuntimeError(f"Error loading model: {e}")

# Image transformations with optimized parameters
transform = transforms.Compose([
    transforms.Resize((IMAGE_SIZE, IMAGE_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

def load_ground_truth_labels(csv_path: str) -> Dict[str, str]:
    """Load and process ground truth labels from CSV."""
    try:
        df = pd.read_csv(csv_path)
        return {row["image"]: LABEL_MAP[row["label"]] for _, row in df.iterrows()}
    except Exception as e:
        raise RuntimeError(f"Error loading ground truth labels: {e}")

class TestDataset(Dataset):
    """Optimized dataset class for test images."""
    def __init__(self, test_folder: str, transform=None):
        self.image_paths = glob.glob(os.path.join(test_folder, "**", "*.jpg"), recursive=True)
        if not self.image_paths:
            raise FileNotFoundError(f"No images found in: {test_folder}")
        self.transform = transform
        # Pre-load image names for faster lookup
        self.image_names = [os.path.basename(p) for p in self.image_paths]

    def __len__(self) -> int:
        return len(self.image_paths)

    def __getitem__(self, idx: int) -> Tuple[torch.Tensor, str, str]:
        img_path = self.image_paths[idx]
        img_name = self.image_names[idx]
        try:
            image = Image.open(img_path).convert("RGB")
            if self.transform:
                image = self.transform(image)
            return image, img_path, img_name
        except Exception as e:
            print(f"Error loading image {img_path}: {e}")
            # Return a dummy image if loading fails
            dummy_img = torch.zeros(3, IMAGE_SIZE, IMAGE_SIZE)
            return dummy_img, img_path, img_name

def evaluate_model(
    model: nn.Module,
    test_loader: DataLoader,
    ground_truth: Dict[str, str]
) -> Tuple[float, Dict[str, float], List[Tuple[str, str, str]]]:
    """Evaluate model performance and return metrics and misclassified examples."""
    correct = 0
    total = 0
    y_true = []
    y_pred = []
    misclassified = []

    model.eval()
    with torch.no_grad(), torch.cuda.amp.autocast(enabled=device.type == "cuda"):
        for images, img_paths, img_names in tqdm(test_loader, desc="Evaluating"):
            images = images.to(device)
            outputs = model(images)
            _, preds = torch.max(outputs, 1)
            preds = preds.cpu().numpy()

            for img_name, img_path, pred in zip(img_names, img_paths, preds):
                if img_name not in ground_truth:
                    continue

                true_label = ground_truth[img_name]
                predicted_label = LABEL_MAP[pred + 1]

                y_true.append(REVERSE_LABEL_MAP[true_label])
                y_pred.append(pred)
                total += 1

                if true_label == predicted_label:
                    correct += 1
                else:
                    if len(misclassified) < 5:  # Store only top 5 misclassified
                        misclassified.append((img_path, true_label, predicted_label))

    accuracy = correct / total * 100 if total > 0 else 0

    # Calculate metrics
    metrics = {
        "accuracy": accuracy,
        "precision": precision_score(y_true, y_pred, average="weighted", zero_division=0),
        "recall": recall_score(y_true, y_pred, average="weighted", zero_division=0),
        "f1": f1_score(y_true, y_pred, average="weighted", zero_division=0)
    }

    return metrics, misclassified

def display_predictions(image_paths: List[str], true_labels: List[str], pred_labels: List[str]):
    """Display images with their true and predicted labels."""
    num_images = len(image_paths)
    if num_images == 0:
        print("No images to display.")
        return

    fig, axes = plt.subplots(1, min(5, num_images), figsize=(15, 5))
    if num_images == 1:
        axes = [axes]

    for i, (img_path, true, pred) in enumerate(zip(image_paths, true_labels, pred_labels)):
        try:
            img = Image.open(img_path)
            axes[i].imshow(img)
            axes[i].axis("off")
            axes[i].set_title(f"True: {true}\nPred: {pred}", fontsize=10)
        except Exception as e:
            print(f"Error displaying image {img_path}: {e}")

    plt.tight_layout()
    plt.show()

def main():
    # Load model
    model = load_model(MODEL_PATH)

    # Load ground truth
    ground_truth = load_ground_truth_labels(TEST_LABELS_PATH)
    print(f"Loaded {len(ground_truth)} ground truth labels.")

    # Create test dataset and loader
    test_dataset = TestDataset(TEST_FOLDER, transform=transform)
    test_loader = DataLoader(
        test_dataset,
        batch_size=BATCH_SIZE,
        shuffle=False,
        num_workers=NUM_WORKERS,
        pin_memory=True  # Faster data transfer to GPU
    )
    print(f"Total test images found: {len(test_dataset)}")

    # Evaluate
    metrics, misclassified = evaluate_model(model, test_loader, ground_truth)

    # Print results
    print("\nModel Evaluation Metrics:")
    print(f"Accuracy: {metrics['accuracy']:.2f}%")
    print(f"Precision: {metrics['precision']:.4f}")
    print(f"Recall: {metrics['recall']:.4f}")
    print(f"F1-Score: {metrics['f1']:.4f}")

    # Display misclassified images
    if misclassified:
        img_paths, true_labels, pred_labels = zip(*misclassified)
        display_predictions(img_paths, true_labels, pred_labels)
    else:
        print("No misclassified images found.")

if __name__ == "__main__":
    main()